In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab4App").getOrCreate()

In [2]:
from functools import reduce

from pyspark.sql.functions import col, sum as spark_sum

Charger le fichier products.csv. En utilisant pyspark, repérer les valeurs aberrantes (ordre de grandeur : plusieurs centaines).

In [3]:
product_df = spark.read.csv("../../data/products.csv", header=True, inferSchema=True)

In [4]:
product_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- code: double (nullable = true)
 |-- fat_100g: double (nullable = true)
 |-- saturated-fat_100g: double (nullable = true)
 |-- sugars_100g: double (nullable = true)
 |-- fiber_100g: double (nullable = true)
 |-- proteins_100g: double (nullable = true)
 |-- salt_100g: double (nullable = true)
 |-- sodium_100g: double (nullable = true)
 |-- autre: double (nullable = true)



In [5]:
product_df.toPandas()

,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
0,0,3.087000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000
1,1,4.530000e+03,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000
2,2,4.559000e+03,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500
3,3,1.608700e+04,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372
4,4,1.609400e+04,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000
...,...,...,...,...,...,...,...,...,...,...
320767,320767,9.948283e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000
320768,320768,9.956745e+07,0.00,0.00,0.00,0.0,0.00,0.00000,0.000,100.00000
320769,320769,9.970230e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000
320770,320770,9.980283e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000


In [6]:
product_df.describe().toPandas()

,summary,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
0,count,320772,320749,243891,229554,244971,200886,259922,255510,255463,320772
1,mean,160385.5,3.8241411905082087E33,12.730378571710334,5.129932302769851,16.003483547358233,2.8621109473035657,7.075939659205786,2.0286239493368763,0.7988154570102396,65.86151103976306
2,stddev,92599.04461170212,1.8551307088505311E36,17.578746693167723,8.01423813836071,22.327284404375717,12.86757803197955,8.409054244714131,128.26945441693383,50.50442790314077,32.09102128093654
3,min,0,1.0,0.0,0.0,-17.86,-6.7,-800.0,0.0,0.0,0.0
4,max,320771,1.0325807005530831E39,714.29,550.0,3520.0,5380.0,430.0,64312.8,25320.0,889.38


Codes manquants

In [7]:
missing_code_df = product_df.filter(col("code").isNull())
print(missing_code_df.count())
missing_code_df.toPandas()

23


,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
0,189068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000
1,189103,NaN,NaN,NaN,NaN,NaN,0.137,NaN,NaN,99.863
2,189109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000
3,189119,NaN,NaN,NaN,NaN,NaN,0.122,NaN,NaN,99.878
4,189152,NaN,NaN,NaN,NaN,NaN,0.158,NaN,NaN,99.842
5,189160,NaN,NaN,NaN,NaN,NaN,0.156,NaN,NaN,99.844
6,189162,NaN,NaN,NaN,NaN,NaN,0.158,NaN,NaN,99.842
7,189168,NaN,NaN,NaN,NaN,NaN,0.120,NaN,NaN,99.880
8,189242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000
9,189244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000


Valeurs > 100

In [8]:
cols_to_check = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "autre",
]    

In [9]:
tol = 2
threshold = 100 + tol

TypeError: DataFrame.filter() missing 1 required positional argument: 'condition'

In [ ]:
from pyspark.sql.functions import lit
more_than_100_condition = reduce(
    lambda x, y: x | (col(y) > 100),
    cols_to_check,
    lit(False),
)
product_df.filter(more_than_100_condition).toPandas()

In [ ]:
more_than_100_condition = reduce(
    lambda x, y: x | y,
    [col(col_to_check) > threshold for col_to_check in cols_to_check],
)

In [ ]:
product_df.filter(more_than_100_condition).toPandas()

Valeurs négatives

In [ ]:
negative_condition = reduce(
    lambda x, y: x | y,
    [col(col_to_check) <0 for col_to_check in cols_to_check],
)

In [ ]:
product_df.filter(negative_condition).toPandas()

Saturated fat et fat

In [ ]:
product_df.filter(col("saturated-fat_100g") > col("fat_100g")).toPandas()

Non proportionnalité sel et sodium

In [38]:
M_NaCl = 58.44
M_Na = 22.99
coef = M_NaCl / M_Na
tol = 0.05
coef_inf = coef - tol
coef_sup = coef + tol

In [39]:
product_df.filter(
    (col("salt_100g") < coef_inf * col("sodium_100g"))
    |
    (col("salt_100g") > coef_sup * col("sodium_100g"))
).toPandas()

,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre


Somme différente de 100

In [5]:
cols_to_sum = [
    "fat_100g",
    #"saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    #"sodium_100g",
    "autre",
]    

In [11]:
tol = 2

In [13]:
product_df = product_df.withColumn("somme", sum(col(col_name) for col_name in cols_to_sum))

In [14]:
product_df.toPandas()

,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,somme
0,0,3.087000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
1,1,4.530000e+03,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000,71.430
2,2,4.559000e+03,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500,99.750
3,3,1.608700e+04,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372,94.158
4,4,1.609400e+04,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
320767,320767,9.948283e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
320768,320768,9.956745e+07,0.00,0.00,0.00,0.0,0.00,0.00000,0.000,100.00000,100.000
320769,320769,9.970230e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
320770,320770,9.980283e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN


In [18]:
product_df.filter(col("somme") < 100 - tol).toPandas()

,_c0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,somme
0,1,4.530000e+03,28.57,28.57,14.29,3.6,3.57,0.00000,0.000000,21.400000,71.430000
1,3,1.608700e+04,57.14,5.36,3.57,7.1,17.86,1.22428,0.482000,7.263720,94.158000
2,7,1.612400e+04,18.75,4.69,15.62,9.4,14.06,0.13970,0.055000,37.285300,95.255000
3,12,1.687200e+04,36.67,5.00,3.33,6.7,16.67,1.60782,0.633000,29.389180,94.367000
4,16,1.805000e+04,60.71,3.57,3.57,10.7,14.29,0.01016,0.004000,7.145840,96.426000
...,...,...,...,...,...,...,...,...,...,...,...
90513,320652,9.556587e+12,15.00,9.00,5.10,6.5,6.00,0.90000,0.354331,57.145669,90.645669
90514,320656,9.557129e+12,92.00,39.00,0.00,0.0,0.00,0.00000,0.000000,0.000000,92.000000
90515,320663,9.560880e+11,10.53,4.39,15.79,8.8,12.28,0.04572,0.018000,48.146280,95.592000
90516,320692,9.609251e+07,53.30,3.80,5.30,0.5,1.70,1.75006,0.689000,32.960940,95.511000


In [17]:
product_df.filter(col("somme") > 100 + tol).count()

514

In [20]:
product_df.drop("_c0").toPandas()

,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,somme
0,3.087000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
1,4.530000e+03,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000,71.430
2,4.559000e+03,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500,99.750
3,1.608700e+04,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372,94.158
4,1.609400e+04,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000,NaN
...,...,...,...,...,...,...,...,...,...,...
320767,9.948283e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
320768,9.956745e+07,0.00,0.00,0.00,0.0,0.00,0.00000,0.000,100.00000,100.000
320769,9.970230e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
320770,9.980283e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,NaN
